# STL and MTL GP Regression Walkthrough

In [3]:
import sys
sys.path.append('../')
#from design import ModelTraining
from datasets import SyntheticData as SD
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from time import time
import matplotlib.pyplot as plt
import methods.mtl.MTL_GP as MtlGP
import os
import numpy as np
import matplotlib
import seaborn as sns


## Setting Up Datasets

The very first step to running through these Gaussian Process Tutorials is retrieving some data to train our models on. Here we are using the CTRP, GDSC and CCLE datasets mentioned in the introduction.

In [4]:
import importlib
importlib.reload(MtlGP)
dataset = SD.SyntheticDataCreator(num_tasks=3,cellsPerTask=400, drugsPerTask=10, function="cosine",
             normalize=False, noise=1, graph=False, test_split=0.3)
dataset.prepare_data()

## Single Task Gaussian Process Example

below is an exaple of training and testing a basic Sparse Gaussian Process from gpytorch with our data.

In [ ]:
import methods.regressor.SparseGP as SGP
importlib.reload(SGP)

y_pred = {}
sparsegp = SGP.SparseGPRegression(num_iters=50, length_scale=50, noise_covar=1.5, n_inducing_points=250)
for k in dataset.datasets:
    sparsegp.fit(dataset.data['train']['x'][k],
               y=dataset.data['train']['y'][k],
               cat_point=dataset.cat_point)
    y_pred[k] = sparsegp.predict(dataset.data['test']['x'][k])
    
for name in y_pred.keys():
    rmse = np.sqrt(np.sum(((y_pred[name] - dataset.data['test']['y'][name]) ** 2) / len(y_pred[name])))
    print(rmse, name)

Next, we have a more complex method, composite kernel Gaussian Process Regression

In [ ]:
import methods.regressor.SparseGPCompositeKernel as sgpc
importlib.reload(sgpc)
y_pred = {}
sparsegpcomp = sgpc.SparseGPCompositeKernelRegression(num_iters=10, length_scale_cell=100, length_scale_drug=100, noise_covar=1.5, n_inducing_points=500, learning_rate=.1)
for k in dataset.datasets:
    sparsegpcomp.fit(dataset.data['train']['x'][k],
               y=dataset.data['train']['y'][k],
               cat_point=dataset.cat_point)
    y_pred[k] = sparsegpcomp.predict(dataset.data['test']['x'][k])
    
for name in y_pred.keys():
    rmse = np.sqrt(np.sum(((y_pred[name] - dataset.data['test']['y'][name]) ** 2) / len(y_pred[name])))
    print(rmse, name)

## Multitask Background

Given a set of observations $y_0$ we wish to learn parameters $\theta_x$ and $k^x$ of the matrix $K_f$. $k^x$ is a covariance function over the inputs and $\theta_x$ are the parameters for that specific covariance function

## Hadamard Product MTL 
A clear limitation of the last method is that although it is technically multitask, it will fail to capture most task relationships. In order to do this I'll introduce another spin on vanilla GP Regression.

Now we just have one model parameterized as:
\begin{align*} 
y_{i} &= f(x_i) + \varepsilon_{i} \\
f &\sim \mathcal{GP}(C_t,K_{\theta}) \\ 
\theta &\sim p(\theta) \\ 
\varepsilon_{i} &\stackrel{iid}{\sim} \mathcal{N}(0, \sigma^2)  \ 
\end{align*}

With one key difference. Our kernel is now defined as: $K([x,i],[x',j]) = k_{inputs}(x,x') * k_{tasks}(i,j)$ where $ k_{tasks} $ is an "index kernel", essentially a lookup table for inter-task covariance. This lookup table is defined $\forall \ i,j \in$ the set of tasks $T$. Here's a basic example with 4 datapoints and 2 tasks.




In [ ]:
importlib.reload(MtlGP)

hadamardMTL = MtlGP.HadamardMTL(num_iters=300, length_scale=20, noise_covar=.24, n_inducing_points=500, \
                                composite=False, learning_rate=.07, validate=False,bias=False,stabilize=False)


hadamardMTL.fit(dataset.data['train']['x'],
                                   y=dataset.data['train']['y'],
                                   catpt=dataset.cat_point)



In [ ]:
y_pred = hadamardMTL.predict(dataset.data['test']['x']) 
for name in y_pred.keys():
    rmse = np.sqrt(np.sum(((y_pred[name].numpy() - dataset.data['test']['y'][name]) ** 2) / len(y_pred[name])))
    print(rmse, name)

## Example Visualizing Covariance Using Getter

In [ ]:
full_covar = hadamardMTL.model.getCovar().numpy()
plt.imshow(full_covar)
plt.imshow(hadamardMTL.model.getCovar().numpy())


In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
fig, ax = plt.subplots()
task_covar = hadamardMTL.model.getTaskCovar().numpy() # cast from torch to numpy
im = ax.imshow(task_covar, cmap="Reds")
ax.set_xticks([200,800,1300])
ax.set_xticklabels(dataset.datasets)
ax.set_yticks([200,800,1300])
ax.set_yticklabels(dataset.datasets)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="10%", pad=0.5)
cbar = plt.colorbar(im, cax = cax)

## Full Multitask GP with Multitask Kernel

In [ ]:
importlib.reload(MtlGP)

gpymtl = MtlGP.GPyFullMTL(num_iters=300, length_scale=15, noise_covar=1, n_inducing_points=200,  num_tasks=3, learning_rate=.05)


gpymtl.fit(dataset.data['train']['x'],
                                   y=dataset.data['train']['y'],
                                   cat_point=dataset.cat_point)


In [10]:
y_pred = gpymtl.predict(dataset.data['test']['x']) 
i = 0
for name in y_pred.keys():
    rmse = np.sqrt(np.sum(((y_pred[name] - dataset.data['test']['y'][name]) ** 2) / len(y_pred[name])))
    i +=  1
    print(rmse, name)

0.5526002760370554 0
0.721262580126851 1
0.7105683397091712 2


## Example Find Initial Conditions

In order to understand what parameters to start at, we can test different configurations of initial conditions

In [ ]:
import importlib
importlib.reload(MtlGP)
multiBias = MtlGP.HadamardMTL(num_iters=10, noise_covar=1.5, n_inducing_points=500, multitask_kernel=False)   #testing #0)

multiBias._find_initial_conditions(dataset.data['train']['x'], dataset.data['train']['y'], \
                                   n_restarts=800,n_iters=50, n_inducing_points=500)

(tensor(1.2674, grad_fn=<NegBackward>),
 {'likelihood.noise_covar.noise': 0.7006388902664185,
  'covar_module.lengthscale': 10.444199562072754})